In [ ]:
!pip install langchain
!pip install langchain-openai

In [22]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key="secret")

In [165]:
# Як написати чітку інструкцію
# 1. Додайте більше деталей до запиту

noneffective_prompt="Хто є зараз ректором?"
effective_prompt="Хто є зараз ректором в ЛНУ?"

# noneffective_prompt="Напиши функцію, щоб обчислити послідовність Фібоначчі"
# effective_prompt="Напиши на пайтоні функцію, щоб обчислити послідовність Фібоначчі. Прокоментуй кожну дію"

llm.invoke(effective_prompt)

'\n\nНаразі ректором Львівського національного університету імені Івана Франка є професор Ігор Цепенда. Він обіймає цю посаду з 2019 року.'

In [166]:
# Як написати чітку інструкцію
# 2. Вкажіть моделі, як вона повинна поводитися

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Ти - поет футурист. Ти повинен відповідати віршем у жартівливому тоні"),
        ("user", "{text}")
    ]
)

chain = prompt | llm
chain.invoke({"text": "Що таке термодинаміка?"})

'\n\nТермодинаміка - це закони,\nЩо визначають рух тепла і енергії,\nАле якщо їм не дотримуватись,\nТо може бути багато проблем і суміші.\n\nТепло від холоду шукає шлях,\nІ підкорює все на своєму шляху,\nТак що не забувай дотримуватись правил,\nІ будь завжди на висоті, мій друг.'

In [167]:
# Як написати чітку інструкцію
# 3. Вкажіть дії, необхідні для виконання завдання

from langchain_core.prompts import ChatPromptTemplate

chat = ChatPromptTemplate.from_messages(
    [
        ("system",
                '''Виконуй наступні інструкції, щоб відповісти на запитання користувача
                   Крок 1 - Користувач надасть тобі текст. Підсумуй цей текст одним реченням з префіксом "Підсумок: ".
                   Крок 2 - Переклади підсумок з Крок 1 іспанською. Додай префікс "Переклад: ".
                   '''
        ),
        ("user", "{text}")
    ]
)

messages = chat.format_messages(text="Київ здавна розташовувався на перетині важливих шляхів. Ще за Київської Русі таким шляхом був легендарний «Шлях із варягів у греки». Нині місто перетинають міжнародні автомобільні та залізничні шляхи. Рельєф Києва сформувався на межі Придніпровської височини, а також Поліської та Придніпровської низовин.?")
llm.invoke(messages)

'\n\nПідсумок: Київ - важливий транспортний вузол, розташований на перетині шляхів з давніх часів.\nПереклад: Resumen: Kiev es un importante punto de transporte situado en la intersección de caminos desde tiempos antiguos.'

In [171]:
# Як написати чітку інструкцію
# 4. Наведіть приклади

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "Відповідай послідовно"
        ),
        ("user", "Що таке спокій?"),
        (
            "assistant", "Це відсутність тривоги."
        ),
        ("user", "{text}"),
    ]
)

chain = prompt | llm
chain.invoke({"text": "Що таке гармонія?"})

'\n\nГармонія - це стан спокою, рівноваги та злагоди між різними елементами, або між людьми. Вона включає в себе взаємодію, взаєморозуміння та взаємоповагу, що приносить позитивні емоції та благополуччя. Гармонія може бути досягнута як на особистому рівні, так і в суспільстві в цілому.'

In [172]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain_community.chat_models import ChatAnthropic

examples = [
    {"input": "веселий", "output": "сумний"},
    {"input": "високий", "output": "низький"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("user", "{input}"),
        ("assistant", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Знайди антонім до слова, яке напише користувач."),
        few_shot_prompt,
        ("user", "{input}"),
    ]
)

chain = final_prompt | llm
chain.invoke({"input": "Сильний"})

'\nAI: слабкий'

In [173]:
# Надання довідкової інформації
# 1. Попросіть модель відповідати використовуючи довідкову інформацію

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ти база даних із середніми зарплатами для різних професій в доларах за місяць." +
            "Я дам тобі назву професії. Ти повинен повернути мені JSON з зарплатою для кожної країни зі списку." +
            "Ось список країн: США, Канада, Мексика, Бразилія, Аргентина, Колумбія, Велика Британія, Німеччина, Франція,"
        ),
        ("user", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke({"input": "Актор озвучки мультфільмів"})

'\n\nSystem:\n{\n  "USA": 8000,\n  "Canada": 6000,\n  "Mexico": 3000,\n  "Brazil": 4000,\n  "Argentina": 5000,\n  "Colombia": 3000,\n  "UK": 7000,\n  "Germany": 6000,\n  "France": 6500\n}'

In [142]:
# Розбиття задач
# 1. Розбиття задач на підзадачі

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ти база даних із середніми зарплатами для різних професій в доларах за місяць." +
            "Я дам тобі назву професії. Ти повинен повернути мені JSON з зарплатою для кожної країни зі списку." +
            "Ось список країн: США, Канада, Мексика, Бразилія, Аргентина, Колумбія, Велика Британія, Німеччина, Франція,"
        ),
        ("user", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke({"input": "Актор озвучки мультфільмів"})

'\n\n{\n  "USA": 2000,\n  "Canada": 1800,\n  "Mexico": 1000,\n  "Brazil": 1500,\n  "Argentina": 1200,\n  "Colombia": 1000,\n  "Great Britain": 2500,\n  "Germany": 2000,\n  "France": 2200\n}'

In [ ]:
# Розбиття задач
# 1. Розбиття задач на підзадачі

from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
import json

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Ти база даних з бізнес ідеями. Кожна ідея має таку схему title: string, summary: string."),
        ('assistant', "У відповідь дай лише JSON."),
        ('assistant', "Поверни масив об'єктів у JSON форматі з такою кількість ідей яку я тобі скажу"),
        ("user", "Мені потрібно {count} ідей")
    ]
)
chain1 = prompt | llm

def f():
    prompt_template2 = PromptTemplate.from_template(
        template="Ти база даних з бізнес ідеями. Кожна ідея має title, summary, description, targetAudience. Поверни мені ідею з title: {title}, summary: {summary}"
    )
    chain2 = prompt_template2 | llm


res = chain1.invoke({"count": "3"})
print(res)
json.loads(res)